## MVP Engenharia de Dados

**Nome:** Bianca Carvalho Lima

**Matrícula:** 4052025000297

**Dataset Original (de 2020 a 2025):** https://www.gov.br/prf/pt-br/acesso-a-informacao/dados-abertos/dados-abertos-da-prf

**Dataset GitHub:** https://github.com/biaacarvalhoo27/PUCRIO_MVP_eng_dados/tree/main/data

## Objetivo do Trabalho
Problema: A segurança no trânsito é um grande problema de saúde pública e logística no Brasil. Entender os fatores de risco, os horários e locais mais críticos pode embasar políticas públicas e campanhas de conscientização.

## Perguntas de Negócio a serem respondidas:

- Tendência Temporal: Qual a tendência (anual) do número de acidentes?
- Qual dia da semana com a maior concentração de acidentes?
- Fator Climático: Existe uma correlação entre condições climáticas (ex.: chuva) e a ocorrência ou gravidade de acidentes?
- Localização Crítica: Quais são as vias, bairros ou regiões da cidade com maior incidência de acidentes?
- Gravidade do Acidente: Quais tipos de acidente (ex.: colisão, atropelamento) estão mais associados a vítimas fatais?

## Imports

In [0]:
import unicodedata
import re
import pandas as pd
import numpy as np
from functools import reduce
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import lit, regexp_replace, col
from pyspark.sql.types import IntegerType, DoubleType, DateType, StringType, StructField, StructType
from pyspark.sql import functions as F

In [0]:
df_silver_fato = spark.table("lakehouse.silver.fato_acidentes")
df_dim_data = spark.table("lakehouse.silver.dim_data")
df_dim_hora = spark.table("lakehouse.silver.dim_hora")
df_dim_localizacao = spark.table("lakehouse.silver.dim_localizacao")
df_dim_tipo_acidente = spark.table("lakehouse.silver.dim_tipo_acidente")
df_dim_condicao_meteorologica = spark.table("lakehouse.silver.dim_condicao_meteorologica")
df_dim_condicao_pista = spark.table("lakehouse.silver.dim_condicao_pista")
df_dim_condicao_tracado_via = spark.table("lakehouse.silver.dim_condicao_tracado_via")

- Tendência Temporal: Qual a tendência (anual) do número de acidentes?

In [0]:
%sql
SELECT 
    d.ano,
    COUNT(*) as total_acidentes,
    SUM(f.quantidade_mortos) as total_mortos,
    SUM(f.quantidade_feridos) as total_feridos,
    ROUND(AVG(f.quantidade_mortos), 3) as media_mortos_por_acidente,
    ROUND(AVG(f.quantidade_feridos), 2) as media_feridos_por_acidente
FROM lakehouse.silver.fato_acidentes f
RIGHT JOIN lakehouse.silver.dim_data d
    ON f.id_data = d.id_data
WHERE d.mes IS NOT NULL
GROUP BY d.ano
ORDER BY d.ano

ano,total_acidentes,total_mortos,total_feridos,media_mortos_por_acidente,media_feridos_por_acidente
2007,286671,28793,429889,0.1,1.5
2008,322899,32332,483043,0.1,1.5
2009,286497,28723,430648,0.1,1.5
2010,330618,32969,496390,0.1,1.5
2011,346975,34373,519831,0.099,1.5
2012,437593,43921,656507,0.1,1.5
2013,336691,34045,504041,0.101,1.5
2014,303767,31016,456108,0.102,1.5
2015,215213,21630,322932,0.101,1.5
2017,197193,19952,295306,0.101,1.5


- Qual dia da semana com a maior concentração de acidentes?

In [0]:
%sql
CREATE OR REPLACE TABLE lakehouse.gold.acidentes_por_dia_semana AS
SELECT 
    dd.ano,
    dd.mes,
    dd.dia_semana,
    COUNT(*) as total_acidentes
FROM lakehouse.silver.fato_acidentes fa
LEFT JOIN lakehouse.silver.dim_hora dh 
    ON fa.id_hora = dh.id_hora
LEFT JOIN lakehouse.silver.dim_data dd 
    ON fa.id_data = dd.id_data
GROUP BY dd.ano, dd.mes, dd.dia_semana
ORDER BY dd.ano ASC, dd.mes ASC, total_acidentes DESC

num_affected_rows,num_inserted_rows


- Fator Climático: Existe uma correlação entre condições climáticas (ex.: chuva) e a ocorrência ou gravidade de acidentes?

In [0]:
%sql
CREATE OR REPLACE TABLE lakehouse.gold.condicao_meteorologica_gravidade AS
SELECT
    dcm.descricao as condicao_meteorologica,
    COUNT(*) as total_acidentes,
    SUM(fa.quantidade_mortos) as total_mortos,
    SUM(fa.quantidade_feridos) as total_feridos,
    ROUND(AVG(fa.quantidade_mortos), 3) as media_mortos_por_acidente,
    ROUND(AVG(fa.quantidade_feridos), 2) as media_feridos_por_acidente
FROM lakehouse.silver.fato_acidentes fa
LEFT JOIN lakehouse.silver.dim_condicao_meteorologica dcm
    ON fa.id_condicao_meteorologica = dcm.id_condicao_meteorologica
WHERE dcm.descricao IS NOT NULL
GROUP BY dcm.descricao
ORDER BY total_acidentes DESC

num_affected_rows,num_inserted_rows


- Localização Crítica: Quais são as vias, municipios ou regiões da cidade com maior incidência de acidentes?

In [0]:
%sql
CREATE OR REPLACE TABLE lakehouse.gold.localizacao_critica AS
SELECT
    dl.uf,
    dl.municipio,
    dl.br,
    dl.km,
    COUNT(*) as total_acidentes,
    SUM(fa.quantidade_mortos) as total_mortos,
    SUM(fa.quantidade_feridos) as total_feridos
FROM lakehouse.silver.fato_acidentes fa
LEFT JOIN lakehouse.silver.dim_localizacao dl
    ON fa.id_localizacao = dl.id_localizacao
WHERE dl.br IS NOT NULL
GROUP BY dl.uf, dl.municipio, dl.br, dl.km
ORDER BY total_acidentes DESC

num_affected_rows,num_inserted_rows


- Gravidade do Acidente: Quais tipos de acidente (ex.: colisão, atropelamento) estão mais associados a vítimas fatais?

In [0]:
%sql
CREATE OR REPLACE TABLE lakehouse.gold.tipo_acidente_gravidade AS
SELECT
    dta.descricao_tipo as tipo_acidente,
    COUNT(*) as total_acidentes,
    SUM(fa.quantidade_mortos) as total_mortos,
    SUM(fa.quantidade_feridos) as total_feridos,
    ROUND(SUM(fa.quantidade_mortos) / COUNT(*), 3) as media_mortos_por_acidente
FROM lakehouse.silver.fato_acidentes fa
LEFT JOIN lakehouse.silver.dim_tipo_acidente dta
    ON fa.id_tipo_acidente = dta.id_tipo_acidente
WHERE dta.descricao_tipo IS NOT NULL
GROUP BY dta.descricao_tipo
ORDER BY media_mortos_por_acidente DESC, total_mortos DESC

num_affected_rows,num_inserted_rows


In [0]:
df = spark.table("lakehouse.gold.localizacao_critica")
display(df)

uf,municipio,br,km,total_acidentes,total_mortos,total_feridos
RJ,RIO DE JANEIRO,101,322.0,1932,170,2871
RJ,ITABORAI,101,322.0,1932,170,2871
RJ,NITEROI,101,322.0,1932,170,2871
RJ,SAO GONCALO,101,322.0,1932,170,2871
PA,MARITUBA,316,0.0,1336,141,2027
PA,CASTANHAL,316,0.0,1336,141,2027
PA,SANTA MARIA DO PARA,316,0.0,1336,141,2027
PA,BENEVIDES,316,0.0,1336,141,2027
PA,SANTA LUZIA DO PARA,316,0.0,1336,141,2027
PA,BELEM,316,0.0,1336,141,2027
